In [5]:
import os
import json
import snowflake.connector
def get_snowflake_conn_params(profile: str = "default") -> dict:
    """Read Snowflake connection parameters from ~/.snowflake/connection.json."""
    conn_path = os.path.expanduser("~/.snowflake/connection.json")
    with open(conn_path) as f:
        data = json.load(f)
    if profile not in data:
        raise ValueError(f"No connection profile '{profile}' in {conn_path}")
    return data[profile]

def run_snowflake_query(query: str, profile: str = "default") -> list:
    """Run a SQL query on Snowflake and return results as list of dicts."""
    params = get_snowflake_conn_params(profile)
    conn = snowflake.connector.connect(
        user=params["user"],
        password=params["password"],
        account=params["account"],
        warehouse=params.get("warehouse"),
        database=params.get("database"),
        schema=params.get("schema"),
        role=params.get("role")
    )
    try:
        with conn.cursor() as cur:
            cur.execute(query)
            columns = [desc[0] for desc in cur.description]
            rows = cur.fetchall()
            return [dict(zip(columns, row)) for row in rows]
    finally:
        conn.close()

def run_snowflake_insert(insert_sql: str, profile: str = "default") -> int:
    """Run an INSERT statement on Snowflake. Returns number of rows inserted."""
    params = get_snowflake_conn_params(profile)
    conn = snowflake.connector.connect(
        user=params["user"],
        password=params["password"],
        account=params["account"],
        warehouse=params.get("warehouse"),
        database=params.get("database"),
        schema=params.get("schema"),
        role=params.get("role")
    )
    try:
        with conn.cursor() as cur:
            cur.execute(insert_sql)
            conn.commit()
            return cur.rowcount
    finally:
        conn.close()


/opt/homebrew/lib/python3.11/site-packages/snowflake/connector/options.py:104: UserWarning: You have an incompatible version of 'pyarrow' installed (20.0.0), please install a version that adheres to: 'pyarrow<19.0.0; extra == "pandas"'
  warn_incompatible_dep(


In [6]:
run_snowflake_query("SELECT CURRENT_VERSION()", profile="default")

[{'CURRENT_VERSION()': '9.14.2'}]

In [8]:
run_snowflake_query("SELECT * FROM MY_DATABASE.PUBLIC.DOCUMENTS", profile="default")

[{'ID': 1,
  'SOURCE_TYPE': 'url',
  'SOURCE': 'https://example.com',
  'CONTENT': 'This is sample content from a website about technology trends.'},
 {'ID': 2,
  'SOURCE_TYPE': 'pdf',
  'SOURCE': 'document.pdf',
  'CONTENT': 'Extracted text from a PDF discussing machine learning algorithms.'},
 {'ID': 3,
  'SOURCE_TYPE': 'snowflake',
  'SOURCE': 'SELECT * FROM CUSTOMERS',
  'CONTENT': 'Customer comment: Great service and fast delivery!'},
 {'ID': 4,
  'SOURCE_TYPE': 'url',
  'SOURCE': 'https://news.com',
  'CONTENT': 'Article summary: New advancements in AI for 2025.'},
 {'ID': 5,
  'SOURCE_TYPE': 'test',
  'SOURCE': 'test_source',
  'CONTENT': 'Test content for loading into Snowflake.'},
 {'ID': 1,
  'SOURCE_TYPE': 'url',
  'SOURCE': 'https://example.com',
  'CONTENT': 'This is sample content from a website about technology trends.'},
 {'ID': 2,
  'SOURCE_TYPE': 'pdf',
  'SOURCE': 'document.pdf',
  'CONTENT': 'Extracted text from a PDF discussing machine learning algorithms.'},
 {'ID